# Replicability Accross Different Corpora: A Case Study with Software Submissions in TIRA

### Install Dependencies


In [1]:
import sys
!{sys.executable} -m pip install git+https://github.com/tira-io/tira.git@development#subdirectory=python-client
!{sys.executable} -m pip install repro_eval==0.4.0
!{sys.executable} -m pip install git+https://github.com/allenai/ir_datasets.git

  Cloning https://github.com/tira-io/tira.git (to revision development) to /tmp/pip-req-build-n8ef1e9_
  Running command git clone --filter=blob:none --quiet https://github.com/tira-io/tira.git /tmp/pip-req-build-n8ef1e9_
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Resolved https://github.com/tira-io/tira.git to commit 4217559835ad59e9b91c6fc1f2ebca70d123c896
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/allenai/ir_datasets.git to /tmp/pip-req-build-uvp15174
  Running command git clone --filter=blob:none --quiet https://github.com/allenai/ir_datasets.git /tmp/pip-req-build-uvp15174
  Resolved https://github.com/allenai/ir_datasets.git to commit 280164ab597fb43bbb5f97f827ab5fa8e74f780e
  Preparing metadata (setup.py) ... done


### Import modules

In [58]:
from tira.rest_api_client import Client
from tqdm import tqdm
import pandas as pd
import numpy as np
import ir_datasets
from repro_eval.Evaluator import RpdEvaluator, RplEvaluator
from repro_eval.util import arp, arp_scores, print_base_adv, print_simple_line, trim

from pathlib import Path
import glob
import json
from datetime import datetime

### Instantiate the TIRA client and define the task
This evaluation is based on the experiments from the TIRA task "ir-benchmarks".

In [3]:
# Get your API Key at: https://tira.io -> Login -> Admin -> API -> New API Key
tira = Client(api_key='<API-KEY-OMMITTED>')

TASK = 'ir-benchmarks'

### Load all evaluations over all datasets of the given TIRA task
If the file does not already exist locally, the evaluations over each dataset from the TIRA task are downloaded from TIRA and then stored together in a single pandas dataframe. Each row is the evaluation of one retrieval system over one particular dataset. The dataframe will be saved to csv/df_eval.csv and loaded from there, as long as the file exists.

In [4]:
output_file = Path(f'csv/df_eval.csv')

if not output_file.exists():
    df_eval = []
    for dataset in tqdm(tira.datasets(TASK)):
        df_eval += [tira.evaluations(task=TASK, dataset=dataset)]
    df_eval = pd.concat(df_eval)
    
    output_file.parent.mkdir(exist_ok=True, parents=True)
    df_eval.to_csv(output_file, index=False)

else:
    df_eval = pd.read_csv(output_file)

df_eval

,task,dataset,team,run_id,software,is_upload,is_docker,nDCG@10,RR,P@10
0,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-14-10-11,BM25 (tira-ir-starter-pyterrier),False,True,0.510,0.935,0.747
1,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-14-12-44,BM25+RM3 (tira-ir-starter-pyterrier),False,True,0.488,0.920,0.715
2,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-18-22-45,DFR_BM25 (tira-ir-starter-pyterrier),False,True,0.508,0.938,0.741
3,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-18-22-16,DFIC (tira-ir-starter-pyterrier),False,True,0.522,0.920,0.713
4,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-18-22-58,DFRee (tira-ir-starter-pyterrier),False,True,0.453,0.840,0.569
...,...,...,...,...,...,...,...,...,...,...
235,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-20-26-06,ANCE Base Cosine (tira-ir-starter-beir),False,True,0.357,0.602,0.374
236,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-19-46-45,MonoBERT Large (tira-ir-starter-gygaggle),False,True,0.438,0.647,0.478
237,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-20-26-56,ANCE Base Dot (tira-ir-starter-beir),False,True,0.368,0.578,0.388
238,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-18-26-34,MonoT5 Base (tira-ir-starter-gygaggle),False,True,0.451,0.637,0.496


### Clean dataframe from duplicate evaluations

In [5]:

# df_eval = df_eval[df_eval['dataset'] == dataset].sort_values('nDCG@10', ascending=False)
df_eval = df_eval.drop_duplicates(subset=['dataset', 'software'], keep=False)
df_eval

,task,dataset,team,run_id,software,is_upload,is_docker,nDCG@10,RR,P@10
0,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-14-10-11,BM25 (tira-ir-starter-pyterrier),False,True,0.510,0.935,0.747
1,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-14-12-44,BM25+RM3 (tira-ir-starter-pyterrier),False,True,0.488,0.920,0.715
2,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-18-22-45,DFR_BM25 (tira-ir-starter-pyterrier),False,True,0.508,0.938,0.741
3,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-18-22-16,DFIC (tira-ir-starter-pyterrier),False,True,0.522,0.920,0.713
4,ir-benchmarks,antique-test-20230107-training,tira-ir-starter,2023-01-07-18-22-58,DFRee (tira-ir-starter-pyterrier),False,True,0.453,0.840,0.569
...,...,...,...,...,...,...,...,...,...,...
235,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-20-26-06,ANCE Base Cosine (tira-ir-starter-beir),False,True,0.357,0.602,0.374
236,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-19-46-45,MonoBERT Large (tira-ir-starter-gygaggle),False,True,0.438,0.647,0.478
237,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-20-26-56,ANCE Base Dot (tira-ir-starter-beir),False,True,0.368,0.578,0.388
238,ir-benchmarks,wapo-v2-trec-core-2018-20230107-training,tira-ir-starter,2023-02-07-18-26-34,MonoT5 Base (tira-ir-starter-gygaggle),False,True,0.451,0.637,0.496


### Get sets of all datasets and retrieval systems used in the evaluations

In [6]:
DATASETS = df_eval['dataset'].unique()
DATASETS

array(['antique-test-20230107-training',
       'clueweb09-en-trec-web-2009-20230107-training',
       'clueweb09-en-trec-web-2010-20230107-training',
       'clueweb09-en-trec-web-2011-20230107-training',
       'clueweb09-en-trec-web-2012-20230107-training',
       'clueweb12-trec-web-2013-20230107-training',
       'clueweb12-trec-web-2014-20230107-training',
       'cord19-fulltext-trec-covid-20230107-training',
       'cranfield-20230107-training',
       'medline-2004-trec-genomics-2004-20230107-training',
       'msmarco-passage-trec-dl-2019-judged-20230107-training',
       'vaswani-20230107-training',
       'wapo-v2-trec-core-2018-20230107-training'], dtype=object)

In [7]:
SOFTWARES = df_eval['software'].unique()
SOFTWARES

array(['BM25 (tira-ir-starter-pyterrier)',
       'BM25+RM3 (tira-ir-starter-pyterrier)',
       'DFR_BM25 (tira-ir-starter-pyterrier)',
       'DFIC (tira-ir-starter-pyterrier)',
       'DFRee (tira-ir-starter-pyterrier)',
       'DFIZ (tira-ir-starter-pyterrier)',
       'DFReeKLIM (tira-ir-starter-pyterrier)',
       'DirichletLM (tira-ir-starter-pyterrier)',
       'DLH (tira-ir-starter-pyterrier)',
       'DPH (tira-ir-starter-pyterrier)',
       'Hiemstra_LM (tira-ir-starter-pyterrier)',
       'IFB2 (tira-ir-starter-pyterrier)',
       'In_expB2 (tira-ir-starter-pyterrier)',
       'TF_IDF (tira-ir-starter-pyterrier)',
       'XSqrA_M (tira-ir-starter-pyterrier)',
       'InL2 (tira-ir-starter-pyterrier)',
       'PL2 (tira-ir-starter-pyterrier)',
       'LGD (tira-ir-starter-pyterrier)',
       'In_expC2 (tira-ir-starter-pyterrier)',
       'InB2 (tira-ir-starter-pyterrier)',
       'Js_KLs (tira-ir-starter-pyterrier)',
       'BM25 Re-Rank (tira-ir-starter-pyterrier)',
       

### Download the run files of every evaluation
Loads the runs file of each evaluation of the TIRA task from TIRA, if the file does not already exist locally.  Processes the file data to an appropriate format and saves it to runs/{dataset}/run_{dataset}_{software}.txt.

In [8]:
for dataset in tqdm(DATASETS):
    for software in SOFTWARES:
        if not df_eval[(df_eval.dataset == dataset) & (df_eval.software == software)].empty:
            output_file = Path(f'runs/{dataset}/run_{dataset}_{software}.txt')
            if not output_file.exists():
                run = tira.download_run(TASK, dataset, software)
                output_file.parent.mkdir(exist_ok=True, parents=True)
                output_file.write_text(run.to_string(header=False, index=False))

100%|██████████| 13/13 [00:00<00:00, 75.91it/s]


### Map TIRA dataset names to coresponding ir_dataset IDs
A dictionary to map the TIRA format of dataset names to the IDs of ir_datasets.

In [9]:
IR_DATASET_IDS = {
    'antique-test-20230107-training': 'antique/test',
    'clueweb09-en-trec-web-2009-20230107-training': 'clueweb09/en/trec-web-2009',
    'clueweb09-en-trec-web-2010-20230107-training': 'clueweb09/en/trec-web-2010',
    'clueweb09-en-trec-web-2011-20230107-training': 'clueweb09/en/trec-web-2011',
    'clueweb09-en-trec-web-2012-20230107-training': 'clueweb09/en/trec-web-2012',
    'clueweb12-trec-web-2013-20230107-training': 'clueweb12/trec-web-2013',
    'clueweb12-trec-web-2014-20230107-training': 'clueweb12/trec-web-2013',
    'cord19-fulltext-trec-covid-20230107-training': 'cord19/fulltext/trec-covid',
    'cranfield-20230107-training': 'cranfield',
    'medline-2004-trec-genomics-2004-20230107-training': 'medline/2004/trec-genomics-2004',
    'msmarco-passage-trec-dl-2019-judged-20230107-training': 'msmarco-passage/trec-dl-2019/judged',
    'vaswani-20230107-training': 'vaswani',
    'wapo-v2-trec-core-2018-20230107-training': 'wapo/v2/trec-core-2018'
}

### Load qrels via ir_datasets
Downloads the qrels of each dataset with ir_datasets, if the file does not already exists locally. Processes the data to an appropriate format and saves it to qrels/qrels_{dataset}.txt.

In [10]:
def map_qrel(qrel: tuple) -> str:
    iteration = getattr(qrel, 'iteration', 0)
    return f"{qrel.query_id} {iteration} {qrel.doc_id} {qrel.relevance}"

def write_qrels_to_file(qrels, path):
    output_file.parent.mkdir(exist_ok=True, parents=True)
    with output_file.open('wt') as f:
        f.writelines('%s\n' % qrel for qrel in qrels)

for dataset in tqdm(DATASETS):
    output_file = Path(f'qrels/qrels_{dataset}.txt')
    if not output_file.exists():
        ir_ds = ir_datasets.load(IR_DATASET_IDS[dataset])
        qrels = (map_qrel(qrel) for qrel in ir_ds.qrels_iter())
        write_qrels_to_file(qrels, output_file)

100%|██████████| 13/13 [00:00<00:00, 31961.28it/s]


### Function to evaluate the replication of results of retrieval systems on another dataset 
Uses repro_eval to evaluate replicability tests of a tuple of retrieval systems across datasets.  
Is frequently called by the two functions further down to evaluate the dataset wide replicability of retrieval systems.

In [88]:
def evaluate_replicability(qrels_orig, run_b_orig, run_a_orig, run_b_rpl, run_a_rpl, qrels_rpl, measures):
    rpl_eval = RplEvaluator(qrel_orig_path=qrels_orig,
                        run_b_orig_path=run_b_orig,
                        run_a_orig_path=run_a_orig,
                        run_b_rep_path=run_b_rpl,
                        run_a_rep_path=run_a_rpl,
                        qrel_rpl_path=qrels_rpl)
    rpl_eval.trim()
    rpl_eval.evaluate()
    
    er = rpl_eval.er()
    dri = rpl_eval.dri()
    p = rpl_eval.ttest()
    
    metrics = {}

    for measure in measures:
        metrics[measure] = {
            'measure': measure,
            'er': er[measure],
            'dri': dri[measure],
            'p_base': p.get('baseline').get(measure),
            'p_adv': p.get('advanced').get(measure),
        }

    return metrics

Helper functions to write evaluations to files.

In [ ]:
def write_to_csv(timestamp, eval_type, evaluation):
    output_file = Path(f'csv/eval_{eval_type}_{timestamp}.csv')
    output_file.parent.mkdir(exist_ok=True, parents=True)
    
    df = pd.DataFrame(evaluation)
    df = df.transpose()
    df.to_csv(output_file, index=False)


def write_to_json(timestamp, eval_type, evaluation):
    output_file_json = Path(f'json/eval_{eval_type}_{timestamp}.json')
    output_file_json.parent.mkdir(exist_ok=True, parents=True)
    with output_file_json.open('w') as f:
        json.dump(evaluation, f, indent=4)

Values to use for upcoming replicability evaluation processes:

QRELS -> list of paths to all qrels files

RUNS -> list of paths to all run files

MEASURES -> list of all measures to measure in evaluation

DATASET_TEST -> small test set of datasets for testing the evaluation process in a less time consuming process

In [87]:
QRELS = glob.glob('qrels/*.txt')
RUNS = glob.glob('runs/*/*.txt')

MEASURES = [
    'ndcg_cut_10', 
    'P_10', 
    'recip_rank',
]

DATASETS_TEST = np.array([
    'antique-test-20230107-training',
    'cord19-fulltext-trec-covid-20230107-training',
])

### Function to run a full scale evaluation of replicability across all datasets
Evaluates the replicability of every tuple of evaluated retrieval systems on all datasets against all other datersets on which these tuples has been evaluated.

This is a full scale evaluation with n² evaluations in respect to the number of retrieval systems per dataset as well as n² evaluations in respect to the number of datasets.

The results will be written to json and csv.

In [86]:
def full_replicability_evaluation(datasets=DATASETS):
    run_orig_base = ''
    run_orig_adv = ''

    run_rpl_base = ''
    run_rpl_adv = ''

    qrels_orig = ''
    qrels_rpl = ''

    evals = {}

    for dataset in tqdm(datasets):
        # set qrels of target original dataset
        qrels_orig = [x for x in QRELS if dataset in x][0]

        # isolate all evaluations on target original dataset
        df_dataset = df_eval[df_eval['dataset'] == dataset].sort_values('nDCG@10', ascending=False)
        
        # extract list of all softwares of target original dataset
        softwares_dataset = df_dataset['software'].unique()

        # extract list of all datasets for replicatation
        datasets_rpl = np.delete(datasets, datasets==dataset)

        for sw_base in softwares_dataset:
            # set baseline run of target original dataset
            run_orig_base = [x for x in RUNS if dataset in x and sw_base in x][0]

            # extract list of all softwares to measure against
            softwares_advanced = np.delete(softwares_dataset, softwares_dataset==sw_base)

            for sw_adv in softwares_advanced:
                # set advanced run of target original dataset
                run_orig_adv = [x for x in RUNS if dataset in x and sw_adv in x][0]
            
                for dataset_rpl in datasets_rpl:
                    # isolate all evaluations on target replication dataset
                    df_dataset_rpl = df_eval[df_eval['dataset'] == dataset_rpl]

                    # extract list of all softwares of target replication dataset
                    softwares_dataset_rpl = df_dataset_rpl['software'].unique()

                    if sw_base in softwares_dataset_rpl and sw_adv in softwares_dataset_rpl:
                        # set qrels of target original dataset
                        qrels_rpl = [x for x in QRELS if dataset_rpl in x][0]

                        # set baseline run of target replication dataset
                        run_rpl_base = [x for x in RUNS if dataset_rpl in x and sw_base in x][0]

                        # set baseline run of target replication dataset
                        run_rpl_adv = [x for x in RUNS if dataset_rpl in x and sw_adv in x][0]

                        metrics = evaluate_replicability(qrels_orig, run_orig_base, run_orig_adv, run_rpl_base, run_rpl_adv, qrels_rpl, MEASURES)

                        for measure in MEASURES:
                            evals[index] = {
                                'dataset_orig': dataset,
                                'dataset_rpl': dataset_rpl,
                                'system_base': softwares_dataset[i],
                                'system_adv': sw_adv,
                                'measure': metrics[measure]['measure'],
                                'er': metrics[measure]['er'],
                                'dri': metrics[measure]['dri'],
                                'p_base': metrics[measure]['p_base'],
                                'p_adv': metrics[measure]['p_adv']
                            }   
                            index += 1


    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
    write_to_csv(timestamp, 'full', evals)
    write_to_json(timestamp, 'full', evals)

    return evals

### Function to run a binomical scale evaluation of replicability across all datasets
Evaluates the replicability of ordered tuples of evaluated retrieval systems on all datasets against all other datersets on which these tuples has been evaluated.

These tuples are ordered by their nDCG@10 value, where one retrieval system functions as a baseline against an advanced retrieval system. 

The scale of this evaluation is binomical in respect to the number of retrieval systems per dataset. It still is in the scale of n² in respect to the number of datasets.

The results will be written to json and csv.

In [89]:
def binomial_replicability_evaluation(datasets=DATASETS):
    run_orig_base = ''
    run_orig_adv = ''
    run_rpl_base = ''
    run_rpl_adv = ''
    qrels_orig = ''
    qrels_rpl = ''

    evals = {}
    index = 0

    # loop through all datasets
    for dataset in tqdm(datasets):
        # set qrels of target original dataset
        qrels_orig = [x for x in QRELS if dataset in x][0]

        # isolate all evaluations on target original dataset
        df_dataset = df_eval[df_eval['dataset'] == dataset].sort_values('nDCG@10', ascending=True)
        
        # extract list of all softwares used on target original dataset
        softwares_dataset = df_dataset['software'].unique()

        # extract list of all datasets for replicatation
        datasets_rpl = np.delete(datasets, datasets==dataset)

        for i in range(len(softwares_dataset)):
            # set baseline run of target original dataset
            run_orig_base = [x for x in RUNS if dataset in x and softwares_dataset[i] in x][0]

            # extract list of all softwares to measure against
            softwares_advanced = softwares_dataset[i+1:]

            for sw_adv in softwares_advanced:
                # set advanced run of target original dataset
                run_orig_adv = [x for x in RUNS if dataset in x and sw_adv in x][0]
            
                for dataset_rpl in datasets_rpl:
                    # isolate all evaluations on target replication dataset
                    df_dataset_rpl = df_eval[df_eval['dataset'] == dataset_rpl]

                    # extract list of all softwares of target replication dataset
                    softwares_dataset_rpl = df_dataset_rpl['software'].unique()

                    if softwares_dataset[i] in softwares_dataset_rpl and sw_adv in softwares_dataset_rpl:
                        # set qrels of target original dataset
                        qrels_rpl = [x for x in QRELS if dataset_rpl in x][0]

                        # set baseline run of target replication dataset
                        run_rpl_base = [x for x in RUNS if dataset_rpl in x and softwares_dataset[i] in x][0]

                        # set baseline run of target replication dataset
                        run_rpl_adv = [x for x in RUNS if dataset_rpl in x and sw_adv in x][0]

                        metrics = evaluate_replicability(qrels_orig, run_orig_base, run_orig_adv, run_rpl_base, run_rpl_adv, qrels_rpl, MEASURES)

                        for measure in MEASURES:
                            evals[index] = {
                                'dataset_orig': dataset,
                                'dataset_rpl': dataset_rpl,
                                'system_base': softwares_dataset[i],
                                'system_adv': sw_adv,
                                'measure': metrics[measure]['measure'],
                                'er': metrics[measure]['er'],
                                'dri': metrics[measure]['dri'],
                                'p_base': metrics[measure]['p_base'],
                                'p_adv': metrics[measure]['p_adv']
                            }   
                            index += 1


    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
    write_to_csv(timestamp, 'binomial', evals)
    write_to_json(timestamp, 'binomial', evals)

    return evals


Start replicability evaluation and print the resulting dataframe.

In [90]:
evals = binomial_replicability_evaluation(DATASETS_TEST)

df = pd.DataFrame(evals)
df = df.transpose()

df

  0%|          | 0/2 [00:00<?, ?it/s]/home/simon/anaconda3/envs/bachelor/lib/python3.10/site-packages/repro_eval/measure/overall_effects.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  yield measure, (mean - mean_scores_b.get(measure)) / mean_scores_b.get(measure)
/home/simon/anaconda3/envs/bachelor/lib/python3.10/site-packages/repro_eval/measure/overall_effects.py:142: RuntimeWarning: invalid value encountered in double_scalars
  yield measure, (mean - mean_scores_b.get(measure)) / mean_scores_b.get(measure)
/home/simon/anaconda3/envs/bachelor/lib/python3.10/site-packages/repro_eval/measure/statistics.py:32: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  yield measure, ttest_ind(scores, topic_scores_rep.get(measure)).pvalue
/home/simon/anaconda3/envs/bachelor/lib/python3.10/site-packages/repro_eval/measure/overall_effects.py:82: RuntimeWa

,dataset_orig,dataset_rpl,system_base,system_adv,measure,er,dri,p_base,p_adv
0,antique-test-20230107-training,cord19-fulltext-trec-covid-20230107-training,DirichletLM (tira-ir-starter-pyterrier),XSqrA_M (tira-ir-starter-pyterrier),ndcg_cut_10,1.649898,-0.020232,0.000052,0.0
1,antique-test-20230107-training,cord19-fulltext-trec-covid-20230107-training,DirichletLM (tira-ir-starter-pyterrier),XSqrA_M (tira-ir-starter-pyterrier),P_10,2.241509,-0.084049,0.001181,0.000001
2,antique-test-20230107-training,cord19-fulltext-trec-covid-20230107-training,DirichletLM (tira-ir-starter-pyterrier),XSqrA_M (tira-ir-starter-pyterrier),recip_rank,0.751698,0.043927,0.890237,0.681235
3,antique-test-20230107-training,cord19-fulltext-trec-covid-20230107-training,DirichletLM (tira-ir-starter-pyterrier),ANCE Base Cosine (tira-ir-starter-beir),ndcg_cut_10,0.926459,0.108797,0.000052,0.000042
4,antique-test-20230107-training,cord19-fulltext-trec-covid-20230107-training,DirichletLM (tira-ir-starter-pyterrier),ANCE Base Cosine (tira-ir-starter-beir),P_10,2.35125,-0.056948,0.001181,0.000004
...,...,...,...,...,...,...,...,...,...
1795,cord19-fulltext-trec-covid-20230107-training,antique-test-20230107-training,In_expB2 (tira-ir-starter-pyterrier),MonoT5 Base (tira-ir-starter-gygaggle),P_10,-5.20202,0.334073,0.253824,0.0
1796,cord19-fulltext-trec-covid-20230107-training,antique-test-20230107-training,In_expB2 (tira-ir-starter-pyterrier),MonoT5 Base (tira-ir-starter-gygaggle),recip_rank,-1.1763,0.203697,0.000122,0.386362
1797,cord19-fulltext-trec-covid-20230107-training,antique-test-20230107-training,In_expC2 (tira-ir-starter-pyterrier),MonoT5 Base (tira-ir-starter-gygaggle),ndcg_cut_10,0.136684,0.076017,0.000222,0.000002
1798,cord19-fulltext-trec-covid-20230107-training,antique-test-20230107-training,In_expC2 (tira-ir-starter-pyterrier),MonoT5 Base (tira-ir-starter-gygaggle),P_10,-4.608586,0.310467,0.566708,0.0
